In [1]:
# summarize existing chor outputs per plate
# Conny Lin | May 27, 2020

In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from pathlib import PurePath
import pathlib
import glob
import pickle

In [3]:
dir_save = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'

get files under all MWT folders in dir_drive

example path: /Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_slo1/VG903_400mM/20190418_141335/VG903_OH_15x3_t96h20C_100s30x10s10s_A_0418_jv410014.png

parse the file names into:
* extension
* filename prefix
* filename suffix (e.g. shanespark)
* mwt name
* group name
* expname
    * exp date
    * tracker
    * experimenter
    * exp condition
        * pre-plate
        * taps
        * ISI
        * post-tap
    * exp name tag
* MWT DB source (e.g. MWT, MWT bad)

In [4]:
# get all unique suffix from all file paths

In [5]:
# load all file paths
filename = os.path.join(dir_save,'allfilepaths.pickle')
with open(filename,'rb') as f:
    filepaths = pickle.load(f)
# make it into dataframe
df_filepaths = pd.DataFrame(filepaths,columns=['filepath'])

In [6]:
# transform files into path obj
file_path_obj = list(map(PurePath,df_filepaths['filepath']))
# get MWT path
df_filepaths['mwtpath'] = np.array(list(map(lambda x: os.path.dirname(x), df_filepaths['filepath'])))
# get suffix
df_filepaths['suffix'] = np.array(list(map(lambda x: x.suffix, file_path_obj)))


In [7]:
df_filenames  = pd.DataFrame(list(map(lambda x: x.name, file_path_obj)),
                             columns=['filenames'])

In [8]:
# get .dat
a = df_filenames['filenames'].str.extract(r'(?<=[.])([A-z]{1,}\d{0,}[.]dat)')
ind_datext = ~a.iloc[:,0].isna()
df_filepaths.loc[ind_datext, 'suffix'] = a.loc[ind_datext,0]

In [9]:
# get xx.000.dat
a = df_filenames['filenames'].str.extract(r'(?<=[.])([A-z]{1,}[.]\d{1,}[.]dat)')
b = a.iloc[:,0].str.replace(r'(?<=[.])(\d{3,})(?=[.]dat)','id')
ind_datext = ~b.isna()
df_filepaths.loc[ind_datext, 'suffix'] = b[ind_datext]

In [10]:
# count suffix by plate
df = df_filepaths.groupby(['mwtpath', 'suffix']).count()
df_suffix_count = df.unstack(level=-1)

In [11]:
df_suffix_count

filepath                    \
suffix                                                      .blob .blobs .csv   
mwtpath                                                                         
/Volumes/COBOLT/MWT DISCARD/20100216X_JC_0s0x0s...      NaN   NaN    1.0  NaN   
/Volumes/COBOLT/MWT DISCARD/20100226X_JC_0s0x0s...      NaN   NaN    1.0  NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...      NaN   NaN    2.0  NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...      NaN   NaN    1.0  NaN   
/Volumes/COBOLT/MWT DISCARD/20110820X_SS_1500s3...      NaN   NaN    1.0  NaN   
...                                                     ...   ...    ...  ...   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...      NaN   NaN    5.0  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...      NaN   NaN    7.0  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...      NaN   NaN    3.0  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...      NaN   NaN    3.0  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...      NaN   NaN    1.0  NaN   

                                                                            \
suffix                                             .dat .db .eps .fig .jpg   
mwtpath                                                                      
/Volumes/COBOLT/MWT DISCARD/20100216X_JC_0s0x0s...  NaN NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT DISCARD/20100226X_JC_0s0x0s...  1.0 NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...  2.0 NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...  1.0 NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT DISCARD/20110820X_SS_1500s3...  NaN NaN  NaN  NaN  NaN   
...                                                 ...  ..  ...  ...  ...   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...  NaN NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...  NaN NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...  NaN NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...  NaN NaN  NaN  NaN  NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...  NaN NaN  NaN  NaN  NaN   

                                                        ...           \
suffix                                              .m  ... evan.dat   
mwtpath                                                 ...            
/Volumes/COBOLT/MWT DISCARD/20100216X_JC_0s0x0s... NaN  ...      NaN   
/Volumes/COBOLT/MWT DISCARD/20100226X_JC_0s0x0s... NaN  ...      NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s... NaN  ...      NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s... NaN  ...      NaN   
/Volumes/COBOLT/MWT DISCARD/20110820X_SS_1500s3... NaN  ...      1.0   
...                                                 ..  ...      ...   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_... NaN  ...      NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_... NaN  ...      NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_... NaN  ...      NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_... NaN  ...      NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_... NaN  ...      NaN   

                                                                   \
suffix                                             evanall.id.dat   
mwtpath                                                             
/Volumes/COBOLT/MWT DISCARD/20100216X_JC_0s0x0s...            NaN   
/Volumes/COBOLT/MWT DISCARD/20100226X_JC_0s0x0s...            NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...            NaN   
/Volumes/COBOLT/MWT DISCARD/20100309X_JC_0s0x0s...            NaN   
/Volumes/COBOLT/MWT DISCARD/20110820X_SS_1500s3...            NaN   
...                                                           ...   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...            NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...            NaN   
/Volumes/COBOLT/MWT/20190418X_XX_100s30x10s10s_...     

In [12]:
# save file dataframe
path_save = os.path.join(dir_save, 'file_summary_mwt.pickle')
with open(path_save,'wb') as f:
    pickle.dump(df_suffix_count,f)

In [13]:
print('complete')

complete
